In [1]:

import os
from pprint import pprint

%load_ext autoreload
%autoreload 2

In [4]:
from pprint import pprint

from farmbase_client import AuthenticatedClient
from farmbase_client.api.contacts import contacts_get_contact
from farmbase_client.api.gaez import gaez_aez_classification


from farmbase_client.api.contacts import contacts_get_contact_by_phone as get_contact_by_phone

import os

pprint(os.environ)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

from farmwise.settings import settings
async def get_contact():
    with AuthenticatedClient(base_url="http://localhost:8000/api/v1", token=settings.FARMBASE_API_KEY) as client:
        print(client.get_async_httpx_client().base_url)
        contact = await get_contact_by_phone.asyncio(
            client=client,
            organization="default",
            phone="31657775781",
        )
        return contact
await get_contact()

environ({'SDKMAN_CANDIDATES_API': 'https://api.sdkman.io/2', 'COMMAND_MODE': 'unix2003', 'PYTHONUNBUFFERED': '1', 'PYENV_ROOT': '/Users/markns/.pyenv', 'SHELL': '/opt/homebrew/bin/fish', 'PYTHONPATH': '/Users/markns/workspace/farmwise:/Users/markns/workspace/farmwise/apps/farmbase/src:/Users/markns/workspace/farmwise/apps/farmwise/src:/Users/markns/workspace/farmwise/apps/farmwise/test:/Users/markns/workspace/farmwise/libs/farmbase-agent-toolkit/src:/Users/markns/workspace/farmwise/libs/farmbase-agent-toolkit/test:/Users/markns/workspace/farmwise/libs/farmbase-client/src:/Users/markns/workspace/farmwise/libs/farmbase-client/test:/Users/markns/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/pydev:/Users/markns/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/jupyter_debug', '__CFBundleIdentifier': 'com.jetbrains.pycharm-EAP', 'TMPDIR': '/var/folders/5d/c4d7d1814b33g1r6_cn3h87w0000gn/T/', 'LC_ALL': 'en_US.UTF-8', 'PYTHONIOENCODING': 'UTF-8', 'PUPPETEER_SKIP_CHROMIU

ContactRead(preferred_form_of_address='Mark', gender=<Gender.MALE: 'male'>, date_of_birth=datetime.date(1979, 3, 6), estimated_age=40, role=<ContactRole.FARMER: 'farmer'>, experience=2, email=None, id=1, name='Mark Nuttall-Smith', phone_number='31657775781', organization=OrganizationRead(name='default', description='Default Farmbase organization.', default=True, id=1, slug='default'), farms=[FarmSummary(farm_name='Rafiki farm', location=Location(latitude=-0.365739, longitude=35.809071), id=1, role=<FarmContactRole.OWNER: 'owner'>)])

In [4]:

token = "fsdfds"

latitude = 0.14743243681250168
longitude = 35.08145448691857

with AuthenticatedClient(base_url="http://127.0.0.1:8000/api/v1", token=token) as client:
    response = await gaez_aez_classification.asyncio(client=client, latitude=latitude, longitude=longitude)
    print(response)

with AuthenticatedClient(base_url="http://127.0.0.1:8000/api/v1", token=token) as client:
    response = await contacts_get_contact.asyncio(client=client, organization='default', farmer_id=1)
    print(response)


ConnectError: All connection attempts failed

In [35]:


import inspect
import json
import types
from typing import Any

from agents import Agent, FunctionTool, ModelBehaviorError, RunContextWrapper, Runner, SpanError, set_default_openai_key
from agents.function_schema import function_schema
from agents.util import _error_tracing
from dotenv import find_dotenv, load_dotenv
from loguru import logger
from pydantic import BaseModel, ValidationError

load_dotenv(find_dotenv())

set_default_openai_key(os.environ["OPENAI_API_KEY"])



# client = AuthenticatedClient(base_url="http://127.0.0.1:8000/api/v1", token=token)


def do_some_work(data: str) -> str:
    return "done"


class FunctionArgs(BaseModel):
    username: str
    age: int


the_func = farmers_get_farmer.asyncio


def copy_function_without_client(func):
    sig = inspect.signature(func)
    parameters = list(sig.parameters.values())

    # Remove the 'client' parameter
    parameters = [p for p in parameters if p.name != "client"]
    new_sig = sig.replace(parameters=parameters)

    def wrapper(*args, **kwargs):
        # Remove 'client' from kwargs if present
        kwargs.pop("client", None)
        return func(*args, **kwargs)

    # Create the new function
    new_func = types.FunctionType(
        wrapper.__code__,
        wrapper.__globals__,
        name=f"{func.__name__}",
        argdefs=wrapper.__defaults__,
        closure=wrapper.__closure__
    )

    # Set the new signature
    new_func.__signature__ = new_sig
    new_func.__doc__ = func.__doc__

    return new_func


failure_error_function = None

schema = function_schema(
    func=copy_function_without_client(the_func),
    # name_override=name_override,
    # description_override=description_override,
    # docstring_style=docstring_style,
    # use_docstring_info=use_docstring_info,
    # strict_json_schema=strict_mode,
)



async def _on_invoke_tool_impl(ctx: RunContextWrapper[Any], input: str) -> Any:
    try:
        json_data: dict[str, Any] = json.loads(input) if input else {}
    except Exception as e:
        logger.debug(f"Invalid JSON input for tool {schema.name}: {input}")
        raise ModelBehaviorError(
            f"Invalid JSON input for tool {schema.name}: {input}"
        ) from e

    logger.debug(f"Invoking tool {schema.name} with input {input}")

    try:
        parsed = (
            schema.params_pydantic_model(**json_data)
            if json_data
            else schema.params_pydantic_model()
        )
    except ValidationError as e:
        raise ModelBehaviorError(f"Invalid JSON input for tool {schema.name}: {e}") from e

    args, kwargs_dict = schema.to_call_args(parsed)

    logger.debug(f"Tool call args: {args}, kwargs: {kwargs_dict}")


    with AuthenticatedClient(base_url="http://127.0.0.1:8000/api/v1", token=token) as client:
        result = await the_func(client=client, *args, **kwargs_dict)

    # if inspect.iscoroutinefunction(the_func):
    #     if schema.takes_context:
    #         result = await the_func(ctx, *args, **kwargs_dict)
    #     else:
    #         result = await the_func(*args, **kwargs_dict)
    # else:
    #     if schema.takes_context:
    #         result = the_func(ctx, *args, **kwargs_dict)
    #     else:
    #         result = the_func(*args, **kwargs_dict)

    logger.debug(f"Tool {schema.name} returned {result}")

    return result

async def _on_invoke_tool(ctx: RunContextWrapper[Any], input: str) -> Any:
    try:
        return await _on_invoke_tool_impl(ctx, input)
    except Exception as e:
        if failure_error_function is None:
            raise

        result = failure_error_function(ctx, e)
        if inspect.isawaitable(result):
            return await result

        _error_tracing.attach_error_to_current_span(
            SpanError(
                message="Error running tool (non-fatal)",
                data={
                    "tool_name": schema.name,
                    "error": str(e),
                },
            )
        )
        return result

tool = FunctionTool(
    name=schema.name,
    description=schema.description,
    params_json_schema=schema.params_json_schema,
    on_invoke_tool=_on_invoke_tool,
    strict_json_schema=True,
)


agent = Agent(
    name="Farmbase agent",
    instructions="You are a helpful agent for interacting with the farmbase API.",
    tools=[tool],
)


print(agent.tools)

async def main():
    result = await Runner.run(agent, input="Please get farmer 1 from the farmbase API with the default organization")
    print(result.final_output)
    # The weather in Tokyo is sunny.


await main()

#
# function_tool(f)

[FunctionTool(name='asyncio', description='Get a farmer.\n\n Get a farmer.', params_json_schema={'properties': {'organization': {'description': '', 'title': 'Organization', 'type': 'string'}, 'farmer_id': {'description': '', 'title': 'Farmer Id', 'type': 'integer'}}, 'required': ['organization', 'farmer_id'], 'title': 'asyncio_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function _on_invoke_tool at 0x11e0d4c20>, strict_json_schema=True)]


2025-05-07 12:43:20.705 | DEBUG    | __main__:_on_invoke_tool_impl:90 - Invoking tool asyncio with input {"organization":"default","farmer_id":1}
2025-05-07 12:43:20.706 | DEBUG    | __main__:_on_invoke_tool_impl:103 - Tool call args: ['default', 1], kwargs: {}
2025-05-07 12:43:21.067 | DEBUG    | __main__:_on_invoke_tool_impl:120 - Tool asyncio returned name='John' phone_number='+31657775782' id=1 created_at=datetime.datetime(2025, 5, 6, 10, 12, 1, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 5, 6, 10, 12, 1, tzinfo=TzInfo(UTC))


Here is the information for Farmer 1 from the default organization:

- **Name:** John
- **Phone Number:** +31657775782
- **ID:** 1
- **Created At:** May 6, 2025, 10:12:01 UTC
- **Updated At:** May 6, 2025, 10:12:01 UTC


In [26]:
async def asyncio(
    organization: str,
    farmer_id: int,
):
    ...

In [5]:
from uuid import UUID

from farmbase_client.api.fields import fields_create_field
from farmbase_client.models import FieldCreate
from geojson_pydantic import LineString

geojson_feature = {
    "type": "LineString",
    "coordinates": [
        [102.0, 0.0], [103.0, 1.0], [104.0, 0.0], [105.0, 1.0]
    ]
}
linestring = LineString(**geojson_feature)
print(linestring)

field = FieldCreate(name="big field",
                    boundary=linestring)
print(field)
with AuthenticatedClient(base_url="http://127.0.0.1:8000", token=token) as client:
    response = fields_create_field.sync(client=client, body=field,
                                        farm_id=UUID('b3d366ef-6398-4f1e-8cf4-566bf216fcba'))

response

bbox=None type='LineString' coordinates=[Position2D(longitude=102.0, latitude=0.0), Position2D(longitude=103.0, latitude=1.0), Position2D(longitude=104.0, latitude=0.0), Position2D(longitude=105.0, latitude=1.0)]
name='big field' area=10.5 soil_type='clay' boundary=LineString(bbox=None, type='LineString', coordinates=[Position2D(longitude=102.0, latitude=0.0), Position2D(longitude=103.0, latitude=1.0), Position2D(longitude=104.0, latitude=0.0), Position2D(longitude=105.0, latitude=1.0)])


FieldPublic(name='big field', area=None, soil_type=None, id=UUID('13587135-d7a1-4d49-a56d-c357af475463'), farm_id=UUID('b3d366ef-6398-4f1e-8cf4-566bf216fcba'), boundary=Polygon(bbox=None, type='Polygon', coordinates=[[Position2D(longitude=102.0, latitude=0.0), Position2D(longitude=103.0, latitude=1.0), Position2D(longitude=104.0, latitude=0.0), Position2D(longitude=105.0, latitude=1.0), Position2D(longitude=102.0, latitude=0.0)]]))

In [1]:
import pandas as pd

df = pd.read_csv("apps/farmbase/data/kalro/suitability.csv")

In [3]:
df.to_pickle("apps/farmbase/data/kalro/suitability.pkl")

In [32]:
df[df.crop == 'Kale']

,crop,variety,dhis2_id,suitability


In [16]:
df[df.crop == 'Barley'].pivot(index='dhis2_id', columns='variety', values='suitability')

variety,ALICIANA,Ahadi,BAHATI,Bima,CERISE LAUREL,COCKTAIL,GRACE,KARNE,NFC TIPPLE,NGAO,PEBLICAN,Planet,QUENCH,Sabini,Tumaini
dhis2_id,,,,,,,,,,,,,,,
A34NiI4rgnf,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
A6Sj8RumZ0m,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN
ABuzigW8Lzw,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
ABy8CNqf2e7,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN
AFIlzKNxXST,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ylWg6VvADJE,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
zCCIu1Vi3Wh,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
zI6vnsXresW,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
import httpx

r = httpx.get("http://127.0.0.1:8000/api/v1/crop-varieties/suitability?crop_type=kale"
              "&latitude=-1.0849843025208&longitude=35.878883361816")
r.json()

{'detail': "No crops found matching 'kale'. Did you mean one of these: Kales, Barley?"}